## Chapter 17: Clean Architecture & Domain‑Driven Design

As applications grow, maintaining a clear separation of concerns becomes critical. Without a well‑defined architecture, codebases tend to become tangled, making changes difficult and bug‑prone. **Clean Architecture** (also known as Onion Architecture or Hexagonal Architecture) is a proven approach that organizes your code into concentric layers, with the core domain at the center. **Domain‑Driven Design (DDD)** provides principles and patterns for modeling complex business domains. In this chapter, you’ll learn how to structure an ASP.NET Core application following Clean Architecture, how to separate your domain from infrastructure concerns, and how to apply DDD patterns like entities, value objects, aggregates, and domain services. By the end, you’ll be able to build maintainable, testable, and business‑focused applications that can evolve over time.

### 17.1 The Onion / Clean Architecture Layers

Clean Architecture organizes software into layers, with dependencies pointing inward. The core contains the business logic and domain model, while outer layers contain infrastructure details like databases, user interfaces, and external services.

A typical Clean Architecture for an ASP.NET Core application consists of four main projects:

1. **Domain** (Core) – Contains entities, value objects, aggregates, domain services, and domain interfaces (e.g., repository interfaces). Has no external dependencies.
2. **Application** (Use Cases) – Contains application logic, DTOs, command/query handlers, and interfaces that the infrastructure layer implements. Depends only on Domain.
3. **Infrastructure** – Implements persistence (e.g., Entity Framework Core), email sending, file storage, etc. Depends on Application (to implement its interfaces) and Domain.
4. **Presentation** (Web) – ASP.NET Core MVC or API project. Depends on Application and Infrastructure.

This separation ensures that business rules are not polluted by infrastructure concerns, and you can swap out implementations (e.g., change databases) without affecting the core logic.

#### Visualizing the Layers

```
┌─────────────────────────┐
│     Presentation        │  (Web UI / API)
│     (Controllers)       │
└───────────┬─────────────┘
            ▼
┌─────────────────────────┐
│      Application        │  (Use Cases, DTOs)
│      (Services)         │
└───────────┬─────────────┘
            ▼
┌─────────────────────────┐
│        Domain           │  (Entities, Value Objects, Interfaces)
│       (Core)            │
└───────────┬─────────────┘
            ▼
┌─────────────────────────┐
│     Infrastructure      │  (EF Core, File I/O, Email)
└─────────────────────────┘
```

**Dependency rule:** Source code dependencies must point inward. The Domain layer has no knowledge of outer layers. Application knows Domain but not Infrastructure. Infrastructure knows both Application and Domain (to implement their interfaces). Presentation knows Application and Infrastructure (to compose the app).

#### Project Structure

In a solution, you might have:

- `MyApp.Domain` – Class library (no framework dependencies).
- `MyApp.Application` – Class library (references Domain).
- `MyApp.Infrastructure` – Class library (references Application and Domain; may reference EF Core, etc.).
- `MyApp.Web` – ASP.NET Core project (references Application and Infrastructure).

### 17.2 Separating Concerns: Why the Web Project Shouldn’t Directly Reference EF Core

In traditional ASP.NET Core applications, you often see `DbContext` directly used in controllers. This couples your presentation layer to the data access technology. If you decide to switch from SQL Server to PostgreSQL, or from EF Core to Dapper, you have to change every controller.

Clean Architecture solves this by placing data access interfaces in the **Domain** or **Application** layer, and their implementations in **Infrastructure**. Controllers depend only on **Application** services (or repositories interfaces). They never reference EF Core directly.

#### Example: Repository Interface in Domain

```csharp
// In Domain project
namespace MyApp.Domain.Repositories
{
    public interface IProductRepository
    {
        Task<Product> GetByIdAsync(int id);
        Task<IEnumerable<Product>> GetAllAsync();
        void Add(Product product);
        void Update(Product product);
        void Remove(Product product);
    }
}
```

#### Implementation in Infrastructure

```csharp
// In Infrastructure project
using MyApp.Domain.Entities;
using MyApp.Domain.Repositories;
using Microsoft.EntityFrameworkCore;

namespace MyApp.Infrastructure.Repositories
{
    public class ProductRepository : IProductRepository
    {
        private readonly AppDbContext _context;

        public ProductRepository(AppDbContext context)
        {
            _context = context;
        }

        public async Task<Product> GetByIdAsync(int id)
        {
            return await _context.Products.FindAsync(id);
        }

        // ... other methods
    }
}
```

#### Using the Repository in Application Layer

```csharp
// In Application project
using MyApp.Domain.Repositories;

namespace MyApp.Application.Services
{
    public class ProductService
    {
        private readonly IProductRepository _productRepository;

        public ProductService(IProductRepository productRepository)
        {
            _productRepository = productRepository;
        }

        public async Task<ProductDto> GetProductAsync(int id)
        {
            var product = await _productRepository.GetByIdAsync(id);
            if (product == null) return null;
            return new ProductDto
            {
                Id = product.Id,
                Name = product.Name,
                Price = product.Price
            };
        }
    }
}
```

#### Controller in Presentation

```csharp
// In Web project
using MyApp.Application.Services;
using Microsoft.AspNetCore.Mvc;

[ApiController]
[Route("api/[controller]")]
public class ProductsController : ControllerBase
{
    private readonly ProductService _productService;

    public ProductsController(ProductService productService)
    {
        _productService = productService;
    }

    [HttpGet("{id}")]
    public async Task<ActionResult<ProductDto>> GetProduct(int id)
    {
        var product = await _productService.GetProductAsync(id);
        if (product == null) return NotFound();
        return Ok(product);
    }
}
```

The Web project has no reference to EF Core or Infrastructure. It only knows Application (and Domain transitively). This makes it easy to test controllers with mocked services.

### 17.3 Introduction to MediatR and the CQRS Pattern

As applications become more complex, the **Command Query Responsibility Segregation (CQRS)** pattern helps separate read and write operations. Commands change state; queries retrieve data. This separation allows you to optimize each side independently (e.g., using different databases or caching strategies).

**MediatR** is a popular library that implements the mediator pattern, helping you implement CQRS by decoupling request handling from controllers. You send a command or query to MediatR, and it dispatches it to the appropriate handler.

#### Installing MediatR

```bash
dotnet add package MediatR
dotnet add package MediatR.Extensions.Microsoft.DependencyInjection
```

#### Basic CQRS with MediatR

**Define a Query:**

```csharp
using MediatR;

public class GetProductQuery : IRequest<ProductDto>
{
    public int Id { get; set; }
}
```

**Define a Handler:**

```csharp
public class GetProductQueryHandler : IRequestHandler<GetProductQuery, ProductDto>
{
    private readonly IProductRepository _productRepository;

    public GetProductQueryHandler(IProductRepository productRepository)
    {
        _productRepository = productRepository;
    }

    public async Task<ProductDto> Handle(GetProductQuery request, CancellationToken cancellationToken)
    {
        var product = await _productRepository.GetByIdAsync(request.Id);
        if (product == null) return null;

        return new ProductDto
        {
            Id = product.Id,
            Name = product.Name,
            Price = product.Price
        };
    }
}
```

**Define a Command:**

```csharp
public class CreateProductCommand : IRequest<int>
{
    public string Name { get; set; }
    public decimal Price { get; set; }
}
```

**Handler for Command:**

```csharp
public class CreateProductCommandHandler : IRequestHandler<CreateProductCommand, int>
{
    private readonly IProductRepository _productRepository;

    public CreateProductCommandHandler(IProductRepository productRepository)
    {
        _productRepository = productRepository;
    }

    public async Task<int> Handle(CreateProductCommand request, CancellationToken cancellationToken)
    {
        var product = new Product
        {
            Name = request.Name,
            Price = request.Price
        };
        _productRepository.Add(product);
        await _productRepository.SaveChangesAsync(); // Assume repository has SaveChanges
        return product.Id;
    }
}
```

**Register MediatR in Program.cs:**

```csharp
builder.Services.AddMediatR(cfg => cfg.RegisterServicesFromAssembly(typeof(GetProductQueryHandler).Assembly));
```

**Use in Controller:**

```csharp
[ApiController]
[Route("api/[controller]")]
public class ProductsController : ControllerBase
{
    private readonly IMediator _mediator;

    public ProductsController(IMediator mediator)
    {
        _mediator = mediator;
    }

    [HttpGet("{id}")]
    public async Task<ActionResult<ProductDto>> GetProduct(int id)
    {
        var query = new GetProductQuery { Id = id };
        var result = await _mediator.Send(query);
        if (result == null) return NotFound();
        return Ok(result);
    }

    [HttpPost]
    public async Task<ActionResult<int>> CreateProduct(CreateProductCommand command)
    {
        var id = await _mediator.Send(command);
        return CreatedAtAction(nameof(GetProduct), new { id }, id);
    }
}
```

#### Benefits of MediatR + CQRS

- **Single Responsibility:** Each handler focuses on one use case.
- **Decoupling:** Controllers don’t need to know about services or repositories; they just send messages.
- **Pipeline Behaviors:** You can add cross‑cutting concerns like logging, validation, or transaction handling by implementing `IPipelineBehavior`.
- **Testability:** Handlers are easy to unit test.

#### Example: Validation Behavior with FluentValidation

You can create a pipeline behavior that automatically validates commands before they reach the handler.

```csharp
public class ValidationBehavior<TRequest, TResponse> : IPipelineBehavior<TRequest, TResponse>
    where TRequest : IRequest<TResponse>
{
    private readonly IEnumerable<IValidator<TRequest>> _validators;

    public ValidationBehavior(IEnumerable<IValidator<TRequest>> validators)
    {
        _validators = validators;
    }

    public async Task<TResponse> Handle(TRequest request, RequestHandlerDelegate<TResponse> next, CancellationToken cancellationToken)
    {
        if (_validators.Any())
        {
            var context = new ValidationContext<TRequest>(request);
            var validationResults = await Task.WhenAll(_validators.Select(v => v.ValidateAsync(context, cancellationToken)));
            var failures = validationResults.SelectMany(r => r.Errors).Where(f => f != null).ToList();
            if (failures.Count != 0)
                throw new ValidationException(failures);
        }
        return await next();
    }
}
```

Register it in DI:

```csharp
builder.Services.AddValidatorsFromAssembly(typeof(Program).Assembly);
builder.Services.AddTransient(typeof(IPipelineBehavior<,>), typeof(ValidationBehavior<,>));
```

### 17.4 Using FluentValidation for Complex Validation Logic

While data annotations work for simple validation, they can become messy for complex rules that involve multiple properties or dependencies. **FluentValidation** is a popular library that provides a fluent interface for building strongly‑typed validation rules.

#### Installing FluentValidation

```bash
dotnet add package FluentValidation.AspNetCore
```

#### Creating a Validator

```csharp
using FluentValidation;

public class CreateProductCommandValidator : AbstractValidator<CreateProductCommand>
{
    public CreateProductCommandValidator()
    {
        RuleFor(x => x.Name)
            .NotEmpty().WithMessage("Name is required.")
            .MaximumLength(100).WithMessage("Name must not exceed 100 characters.");

        RuleFor(x => x.Price)
            .GreaterThan(0).WithMessage("Price must be greater than zero.")
            .LessThanOrEqualTo(10000).WithMessage("Price cannot exceed 10000.");
    }
}
```

#### Integrating with ASP.NET Core

In `Program.cs`:

```csharp
builder.Services.AddFluentValidationAutoValidation(); // Auto-validation for controllers
builder.Services.AddValidatorsFromAssemblyContaining<CreateProductCommandValidator>();
```

Now, when a `CreateProductCommand` arrives at an API controller, FluentValidation will automatically validate it and add errors to `ModelState`. You can also manually validate using the validator.

#### Using FluentValidation with MediatR (as shown earlier)

You can combine FluentValidation with MediatR’s pipeline behavior to centralize validation logic, ensuring that commands are valid before reaching the handler.

### 17.5 Domain‑Driven Design Basics

Domain‑Driven Design (DDD) is a methodology for handling complex business domains. It emphasizes collaboration between technical and domain experts, and it provides tactical patterns for modeling the domain.

#### Key DDD Concepts

- **Entity:** An object with a unique identity that persists over time. Its attributes may change, but its identity remains the same. Example: `Product` (identified by `Id`).
- **Value Object:** An immutable object that has no identity and is defined solely by its attributes. Example: `Address` (street, city, zip code). Two addresses with the same values are considered equal.
- **Aggregate:** A cluster of domain objects that can be treated as a single unit. One entity is the aggregate root, which is the only object external objects can reference directly. Example: `Order` (root) contains `OrderLine` items. You interact with the order through the root.
- **Domain Service:** A stateless service that encapsulates business logic that doesn’t naturally fit into an entity or value object. Example: `TaxCalculator` that computes tax based on order and customer location.
- **Repository:** A mechanism for retrieving and storing aggregates. It provides a collection‑like interface for aggregate roots.
- **Domain Event:** Something that happened in the domain that domain experts care about. Other parts of the system can react to it. Example: `OrderPlaced` event triggers email confirmation and inventory updates.

#### Applying DDD in Clean Architecture

In Clean Architecture, the Domain layer contains entities, value objects, aggregates, domain services, and repository interfaces. The Application layer orchestrates use cases, often using domain services and repositories.

#### Example: Order Aggregate

```csharp
// Domain/Entities/Order.cs
public class Order : IAggregateRoot
{
    public int Id { get; private set; }
    public DateTime OrderDate { get; private set; }
    public string CustomerId { get; private set; }
    private readonly List<OrderLine> _orderLines = new();
    public IReadOnlyCollection<OrderLine> OrderLines => _orderLines.AsReadOnly();

    private Order() { } // For EF Core

    public Order(string customerId)
    {
        CustomerId = customerId;
        OrderDate = DateTime.UtcNow;
    }

    public void AddProduct(Product product, int quantity)
    {
        // Business rule: cannot add duplicate product? Quantity must be positive?
        var line = new OrderLine(product.Id, quantity, product.Price);
        _orderLines.Add(line);
    }

    public decimal TotalPrice => _orderLines.Sum(ol => ol.TotalPrice);
}

// Domain/Entities/OrderLine.cs (Value Object / Entity)
public class OrderLine
{
    public int Id { get; private set; } // Might be an entity if it has its own identity, or a value object if it's part of the aggregate.
    public int ProductId { get; private set; }
    public int Quantity { get; private set; }
    public decimal UnitPrice { get; private set; }
    public decimal TotalPrice => Quantity * UnitPrice;

    internal OrderLine(int productId, int quantity, decimal unitPrice)
    {
        ProductId = productId;
        Quantity = quantity;
        UnitPrice = unitPrice;
    }

    private OrderLine() { } // For EF Core
}
```

Notice that the `Order` class encapsulates business rules (like how to add a product) and ensures the aggregate is always in a valid state.

#### Repository Interface for Order

```csharp
public interface IOrderRepository
{
    Task<Order> GetByIdAsync(int id);
    void Add(Order order);
    void Update(Order order);
}
```

#### Domain Service Example

```csharp
public class PricingService
{
    public decimal CalculateTax(Order order, string customerCountry)
    {
        // Complex tax calculation logic
        return order.TotalPrice * 0.1m; // Simplified
    }
}
```

#### Using the Aggregate in a Use Case (Application Layer)

```csharp
public class PlaceOrderCommandHandler : IRequestHandler<PlaceOrderCommand, int>
{
    private readonly IOrderRepository _orderRepository;
    private readonly IProductRepository _productRepository;
    private readonly PricingService _pricingService;

    public PlaceOrderCommandHandler(IOrderRepository orderRepository, IProductRepository productRepository, PricingService pricingService)
    {
        _orderRepository = orderRepository;
        _productRepository = productRepository;
        _pricingService = pricingService;
    }

    public async Task<int> Handle(PlaceOrderCommand request, CancellationToken cancellationToken)
    {
        var order = new Order(request.CustomerId);
        foreach (var item in request.Items)
        {
            var product = await _productRepository.GetByIdAsync(item.ProductId);
            if (product == null) throw new Exception("Product not found");
            order.AddProduct(product, item.Quantity);
        }

        var tax = _pricingService.CalculateTax(order, request.CustomerCountry);
        // Maybe add tax as a separate line?

        _orderRepository.Add(order);
        await _orderRepository.SaveChangesAsync(); // Assume unit of work

        // Raise domain event (e.g., orderPlacedEvent)
        return order.Id;
    }
}
```

### 17.6 MediatR and CQRS in Depth

We already introduced MediatR. In a Clean Architecture + DDD context, you typically place commands, queries, and their handlers in the **Application** layer. The handlers use domain objects (entities, value objects) and domain services, and they coordinate infrastructure via interfaces (repositories, etc.).

#### Separating Commands and Queries

With CQRS, you might have separate models for reading and writing. For example, a query might return a flattened DTO without involving the full domain model, possibly using Dapper for performance.

**Example of a query handler using Dapper directly (infrastructure):**

```csharp
public class GetProductDetailsQueryHandler : IRequestHandler<GetProductDetailsQuery, ProductDetailsDto>
{
    private readonly DapperConnectionFactory _connectionFactory;

    public GetProductDetailsQueryHandler(DapperConnectionFactory connectionFactory)
    {
        _connectionFactory = connectionFactory;
    }

    public async Task<ProductDetailsDto> Handle(GetProductDetailsQuery request, CancellationToken cancellationToken)
    {
        using var connection = _connectionFactory.Create();
        var sql = "SELECT Id, Name, Price, Description FROM Products WHERE Id = @Id";
        var product = await connection.QueryFirstOrDefaultAsync<ProductDetailsDto>(sql, new { request.Id });
        return product;
    }
}
```

This query bypasses the domain model entirely, which is fine for read operations where you don’t need business rules.

#### Benefits of CQRS with MediatR

- **Scalability:** You can optimize reads and writes separately.
- **Maintainability:** Each use case is a small, focused handler.
- **Flexibility:** You can add new features without affecting existing ones.

### 17.7 Implementing Clean Architecture in ASP.NET Core – Step by Step

Let’s outline the steps to set up a solution following Clean Architecture.

1. **Create a blank solution:**

   ```bash
   dotnet new sln -n MyApp
   ```

2. **Create the projects:**

   ```bash
   dotnet new classlib -n MyApp.Domain
   dotnet new classlib -n MyApp.Application
   dotnet new classlib -n MyApp.Infrastructure
   dotnet new webapi -n MyApp.Web
   ```

3. **Add project references:**

   - Application references Domain.
   - Infrastructure references Application and Domain.
   - Web references Application and Infrastructure.

   ```bash
   dotnet add MyApp.Application reference MyApp.Domain
   dotnet add MyApp.Infrastructure reference MyApp.Application
   dotnet add MyApp.Infrastructure reference MyApp.Domain
   dotnet add MyApp.Web reference MyApp.Application
   dotnet add MyApp.Web reference MyApp.Infrastructure
   ```

4. **Add necessary packages:**

   - Infrastructure: `Microsoft.EntityFrameworkCore.SqlServer`, etc.
   - Application: `MediatR`, `FluentValidation`
   - Web: `MediatR.Extensions.Microsoft.DependencyInjection`, `FluentValidation.AspNetCore`

5. **Define Domain entities and interfaces.**

6. **Implement repositories in Infrastructure.**

7. **Define commands/queries and handlers in Application.**

8. **Configure DI in Web’s `Program.cs`**, registering services from all layers (e.g., using `AddMediatR`, `AddScoped<IOrderRepository, OrderRepository>`, etc.).

#### Dependency Injection in Clean Architecture

Typically, you create extension methods in each layer to register their services.

**In Infrastructure:**

```csharp
public static class DependencyInjection
{
    public static IServiceCollection AddInfrastructure(this IServiceCollection services, IConfiguration configuration)
    {
        services.AddDbContext<AppDbContext>(options =>
            options.UseSqlServer(configuration.GetConnectionString("DefaultConnection")));
        services.AddScoped<IProductRepository, ProductRepository>();
        services.AddScoped<IOrderRepository, OrderRepository>();
        return services;
    }
}
```

**In Application:**

```csharp
public static class DependencyInjection
{
    public static IServiceCollection AddApplication(this IServiceCollection services)
    {
        services.AddMediatR(cfg => cfg.RegisterServicesFromAssembly(typeof(DependencyInjection).Assembly));
        services.AddValidatorsFromAssembly(typeof(DependencyInjection).Assembly);
        services.AddScoped<PricingService>(); // domain service
        return services;
    }
}
```

**In Web’s `Program.cs`:**

```csharp
builder.Services.AddApplication();
builder.Services.AddInfrastructure(builder.Configuration);
```

This keeps the composition root clean and decoupled.

### 17.8 Advantages of Clean Architecture + DDD

- **Testability:** Each layer can be tested in isolation. You can unit test domain logic without any infrastructure.
- **Maintainability:** Business rules are centralized in the Domain layer, not scattered across controllers or services.
- **Flexibility:** Swapping out infrastructure (e.g., changing ORM) is a matter of re‑implementing a few interfaces.
- **Team scalability:** Different teams can work on different layers with minimal conflicts.

### Summary

In this chapter, you learned how to structure an ASP.NET Core application using Clean Architecture and Domain‑Driven Design principles:

- **Clean Architecture** organizes code into concentric layers (Domain, Application, Infrastructure, Presentation) with dependencies pointing inward.
- **Separating concerns** prevents the web project from being tightly coupled to infrastructure concerns like EF Core.
- **MediatR** helps implement the **CQRS pattern**, decoupling request handling and enabling pipeline behaviors.
- **FluentValidation** provides a powerful, fluent way to validate commands and queries.
- **DDD** patterns (entities, value objects, aggregates, domain services) help model complex business domains effectively.

With these patterns, you can build applications that are robust, maintainable, and ready to evolve as business requirements change.

**Exercise:**

1. Refactor your existing project (or a new one) to follow Clean Architecture. Create separate projects for Domain, Application, Infrastructure, and Web.
2. Move your domain models (e.g., `Product`, `Category`) to the Domain project.
3. Define repository interfaces in Domain and implement them in Infrastructure using EF Core.
4. Create a MediatR command and handler for creating a product, and a query for retrieving a product.
5. Add FluentValidation to validate the create command.
6. Ensure that your controllers use MediatR and do not reference Infrastructure directly.

In the next chapter, **"Working with Real‑Time Features (SignalR),"** you’ll learn how to add real‑time functionality to your applications using SignalR. You’ll build live dashboards, chat systems, and notification services that push data from server to clients instantly.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='16. testing_your_application.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='18. working_with_real_time_features.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
